<a href="https://colab.research.google.com/github/Kasaligan/Machine-Learning-UNS/blob/main/TP4/Clase_4_Diagnostico_de_cancer_de_mama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Breast Cancer Diagnostic o Diagnóstico de cáncer de mama

El objetivo de este proyecto es construir un modelo capaz de predecir el diagnóstico de tejidos de cáncer de mama como malignos o benignos. Las características se calculan a partir de una imagen digitalizada de un aspirado con aguja fina (FNA, por sus siglas en inglés) de una masa mamaria. Describen características de los núcleos celulares presentes en la imagen. Distribución de clases: 357 benignas, 212 malignas. Más información sobre este conjunto de datos aquí


In [ ]:
import pandas as pd
cancer_tissues = pd.read_csv("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectBCD/data/data.csv")
cancer_tissues.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


#Objetivo

Para este analisis apunto a implementar el modelo de perceptrones. Claramente no se reinventa la rueda, por lo que utilizo los modulos de sklearn.
El modulo de sklearn que implementa perceptrones ya incluye "boosting", teniendo un parametro para determinar las epocas. Los parametros estan en el siguiente link: 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html

Algo que no implementa el modulo de perceptron por si mismo es el "bagging". Para esto utilizare el modulo ensemble, el cual incluye un metodo que nos hara la vida mas facil. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

Deseo implementar entonces tres diseños:

1.   Perceptrones con una sola epoca).
2.   Perceptrones multi-epoca.
3.   Bagging con perceptrones multi-epoca.

Lo que espero ver es un aumento en la performance a medida que añadimos mas metodos al modelo. 

#CODIGO GENERAL 

Comienzo con el codigo importando librerias de interes. Para algunas de las tareas que realizaba de forma manual en los trabajos anteriores descubri que existen librerias con funciones listas para implementar gracias a este video: https://www.youtube.com/watch?v=oLane_Vh3CU

Por otro lado encontre modulos de sklearn que computan exactitud, precision, f-measure y demas en sklearn.metrics.

In [ ]:
import numpy as np  
from sklearn.model_selection import train_test_split  #modulo para hacer la separacion de datos
from sklearn.linear_model import Perceptron     
from sklearn.ensemble import BaggingClassifier      
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import random 

Pense en filtrar primero los datos que consideraba con distribuciones concentradas, pero considerando que es un examen medico y la informacion brindada es considerada relevante para el analisis lo mejor seria analizar el dataset completo. 

A continuacion aislo la columna "diagnosis" que sera nuestro target. Descarto la columna "ID" que no contiene informacion relevante, y la columna "Unnamed: 32" que dios sabra porque esta ahi. 

In [ ]:
target = cancer_tissues.loc[:,'diagnosis']
data = cancer_tissues.drop(['diagnosis', 'id','Unnamed: 32'], axis=1)

#Para comprobar veo los primeros elementos del dataset
target.head()     
data.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Como el objetivo es comparar el desempeño de los diseños deberan tener los mismos datasets de entrenamiento y testeo. Ademas, incluyo una semilla para que todos partan del mismo lugar y sigan el mismo camino a medida que avanzan las iteraciones. 

In [ ]:
test_size=0.5   #fraccion del dataset que se usa para testeo
[data_train,data_test,target_train,target_test] = train_test_split(data,target,test_size=test_size)
random_state=random.randint(0,1000000)   #Con esto todos los modelos tendran la misma semilla, aunque cambiara en cada ejecucion

##Modelo 1 

A continuacion implemento perceptrones. Ya que el dataset de entrenamiento esta completo no hay batchs, y por lo tanto el numero de epocas es igual al numero de iteraciones. 
Hay muchos argumentos para jugar con la implementacion, pero como desconozco la mayoria me abstengo de copiar sin entender y juego con los mas basicos.

In [ ]:
max_iter=1  #numero de epocas
eta0=1  #constante para multiplicar las updates: mientras menor es mas preciso sera el resultado final pero requerira mas iteraciones para converger

ppn = Perceptron(max_iter=max_iter , eta0=eta0, random_state=random_state)   #creo el modelo
mod_1=ppn.fit(data_train,target_train)    #entreno el modelo
prediccion_1=mod_1.predict(data_test)     #guardo las predicciones del modelo

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


A continuacion evaluo el modelo para ver exactitud, precision y f-measure. 

Para el calculo de precision y f-measure importa saber cual es el caso positivo y cual el negativo. Esto se informa a traves de labels (etiquetas predichas) y pos_label (caso positivo).

In [ ]:
labels=['B','M']  #etiquetas target
pos_label='M'     #target positivo

exact_1=skm.accuracy_score(target_test,prediccion_1)  #calculo exactitud
prec_1=skm.precision_score(target_test,prediccion_1,labels=labels,pos_label=pos_label)  #calculo precision
f_1=skm.f1_score(target_test,prediccion_1,labels=labels,pos_label=pos_label)            #calculo f-measure

print("Exactitud: ",exact_1,"\nPrecision: ",prec_1,"\nF-measure: ",f_1)

Exactitud:  0.8350877192982457 
Precision:  0.7394366197183099 
F-measure:  0.8171206225680935


##MODELO 2

Repito los pasos anteriores con una diferencia: cambio el numero de epocas de 1 a 500. 

Pense en implementar lo siguiente, sin embargo, no lo implemente. Dejo escrito lo que puse antes y al final comento porque no lo hice.

"(...) Para evitar procesamiento innecesario invoco tres parametros extra al modelo: 

1.  *early_stopping* : permite que se interrumpan las epocas antes de llegar al maximo determinado. Para esto separa una fraccion de los datos de entrenamiento para hacer una evaluacion al final de cada epoca. Si la performance no mejora un determinado porcentaje en "n" iteraciones el proceso se detiene y se queda con el ultimo modelo procesado.
2.   *validation_fraction* : porcentaje de los datos de entrenamiento a separar para la validacion.
3.   *tol* : tolerancia que se evalua para interrumpir las iteraciones. No termino de comprender a que se refiere. Segun uno de los resultados de busqueda la tolerancia cambia su significado dependiendo de que modelo se este utilizando (perceptrones, lvm, etc) por lo que dejo este parametro en su valor estandar (aunque esta fuertemente relacionado con la interrupcion de las iteraciones).
4. *n_iter_no_change* : numero de iteraciones en la cual se interrumpe el procedimiento si no mejora el modelo en una cantidad mayor a *tol*. (...) "

La razon por la que decidi no implementarlo es porque el dataset es relativamente pequeño y no creo que una division de datos para una validacion sea lo optimo en este caso. Por lo tanto implementare el numero de iteraciones que deseo sin validacion integrada.


In [ ]:
max_iter=1000  #numero de epocas

#en este caso podria hacer eta0 menor pues tengo mas iteraciones y asi tendre mejores resultados, pero deseo comparar por lo que lo dejo como era

ppn = Perceptron(max_iter=max_iter , eta0=eta0, random_state=random_state)  #creo el modelo
mod_2=ppn.fit(data_train,target_train)    #entreno el modelo 
prediccion_2=mod_2.predict(data_test)     #guardo las predicciones del modelo

A continuacion evaluo el modelo para ver exactitud, precision y f-measure. 


In [ ]:
exact_2=skm.accuracy_score(target_test,prediccion_2)  #calculo exactitud
prec_2=skm.precision_score(target_test,prediccion_2,labels=labels,pos_label=pos_label)  #calculo precision
f_2=skm.f1_score(target_test,prediccion_2,labels=labels,pos_label=pos_label)            #calculo f-measure

print("Exactitud: ",exact_2,"\nPrecision: ",prec_2,"\nF-measure: ",f_2)

Exactitud:  0.8807017543859649 
Precision:  0.813953488372093 
F-measure:  0.860655737704918


##MODELO 3

En este punto cambian un poco las cosas. Para seleccion como clasificador base para el *bagging* un perceptron pero sin random_state. Ya que la idea es que los modelos sean distintos, dejare que la semilla sea determinada aleatoriamente. Luego indico la cantidad de estimadores paralelos que diseñara.

In [ ]:
base_estimator=Perceptron(max_iter=max_iter , eta0=eta0)   #eta0 y max_iter se definieron en el modelo 2 
n_estimators=10       #pondre 10 perceptrones en paralelo

bpnn=BaggingClassifier(base_estimator=base_estimator, n_estimators=n_estimators) #creo el modelo bpnn: bagging perceptron
mod_3=bpnn.fit(data_train,target_train)   #entreno el modelo
prediccion_3=mod_3.predict(data_test)     #guardo las predicciones del modelo

A continuacion evaluo el modelo para ver exactitud, precision y f-measure. 

Para el calculo de precision y f-measure importa saber cual es el caso positivo y cual el negativo. Esto se informa a traves de labels (etiquetas predichas) y pos_label (caso positivo).

In [ ]:
exact_3=skm.accuracy_score(target_test,prediccion_3)  #calculo exactitud
prec_3=skm.precision_score(target_test,prediccion_3,labels=labels,pos_label=pos_label)  #calculo precision
f_3=skm.f1_score(target_test,prediccion_3,labels=labels,pos_label=pos_label)            #calculo f-measure

print("Exactitud: ",exact_3,"\nPrecision: ",prec_3,"\nF-measure: ",f_3)

Exactitud:  0.9157894736842105 
Precision:  0.9690721649484536 
F-measure:  0.8867924528301888


#CONCLUSIONES

Presento los resultados de los tres diseños. Recordemos antes de seguir las caracteristicas de los modelos:
1. El modelo 1 es un diseño por perceptron de una sola epoca.
2. El modelo 2 es un diseño por perceptron de 1000 epocas.
3. El modelo 3 es un diseño "bagging" con base de perceptron que entrena 10 perceptrones de 1000 eras en paralelo y luego entrega una prediccion segun mayoria.

In [ ]:
print("MODELO 1 - Exactitud: ",round(exact_1,5),"\tPrecision: ",round(prec_1,5),"\tF-measure: ",round(f_1,5))
print("MODELO 2 - Exactitud: ",round(exact_2,5),"\tPrecision: ",round(prec_2,5),"\tF-measure: ",round(f_2,5))
print("MODELO 3 - Exactitud: ",round(exact_3,5),"\tPrecision: ",round(prec_3,5),"\tF-measure: ",round(f_3,5))

MODELO 1 - Exactitud:  0.83509 	Precision:  0.73944 	F-measure:  0.81712
MODELO 2 - Exactitud:  0.8807 	Precision:  0.81395 	F-measure:  0.86066
MODELO 3 - Exactitud:  0.91579 	Precision:  0.96907 	F-measure:  0.88679




Luego de probar algunas veces observo que, en general, se cumple en todas las estadisticas que:

Modelo 3 > Modelo 2 > Modelo 1.

Existen algunos casos (raros) en los que el modelo 2 se comporta peor que el modelo 1. Esto podria ser debido al split de los datos aunque no tengo seguridad en esa explicacion. De todas formas el modelo 3 se comporta consistentemente mejor que los otros dos (salvo aún mas raras excepciones).